In [2]:
import pandas as pd
df = pd.read_csv('/content/sample_data/04_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [3]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.0 MB/s eta 0:00:00


In [4]:
%pip install lightbgm

ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [5]:
from lightgbm import LGBMRegressor
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from sklearn.metrics import make_scorer, mean_squared_error

candidate = Catboost(random_state=42)

param_distribs = {
    'iterations': randint(low=6, high=80),
    'depth': randint(low=5, high=25),
}

# Define the scoring function
scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

param_search = RandomizedSearchCV(
    candidate,
    param_distributions=param_distribs,
    n_iter=15,
    random_state=42,
    scoring=scorer,
    cv=3,
    n_jobs=-1
    )

param_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------------------------------------

Learning rate set to 0.5
0:	learn: 1.0844735	total: 230ms	remaining: 12.7s
1:	learn: 1.0750228	total: 399ms	remaining: 10.8s
2:	learn: 1.0715885	total: 549ms	remaining: 9.7s
3:	learn: 1.0677510	total: 700ms	remaining: 9.1s
4:	learn: 1.0661191	total: 871ms	remaining: 8.88s
5:	learn: 1.0655145	total: 1.02s	remaining: 8.5s
6:	learn: 1.0649522	total: 1.17s	remaining: 8.2s
7:	learn: 1.0645241	total: 1.32s	remaining: 7.95s
8:	learn: 1.0643457	total: 1.54s	remaining: 8.06s
9:	learn: 1.0641517	total: 1.85s	remaining: 8.53s
10:	learn: 1.0640480	total: 2.09s	remaining: 8.53s
11:	learn: 1.0639274	total: 2.38s	remaining: 8.73s
12:	learn: 1.0638275	total: 2.67s	remaining: 8.83s
13:	learn: 1.0635871	total: 2.95s	remaining: 8.84s
14:	learn: 1.0633086	total: 3.24s	remaining: 8.85s
15:	learn: 1.0631760	total: 3.51s	remaining: 8.77s
16:	learn: 1.0630877	total: 3.77s	remaining: 8.65s
17:	learn: 1.0629717	total: 4.07s	remaining: 8.59s
18:	learn: 1.0627627	total: 4.38s	remaining: 8.52s
19:	learn: 1.0626518

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7b93e2b0d930>,
                   n_iter=15, n_jobs=-1,
                   param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b9397474580>,
                                        'iterations': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b93e2b0d390>},
                   random_state=42,
                   scoring=make_scorer(mean_squared_error, greater_is_better=False, response_method='predict', squared=False))

In [6]:
param_search.best_params_

{'depth': 7, 'iterations': 56}

In [7]:
best_model = param_search.best_estimator_

In [8]:
from sklearn.metrics import root_mean_squared_error
y_pred = best_model.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0589232796761776

In [9]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [10]:
y_pred = best_model.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0631381820060288

In [ ]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/04_test.csv')
del test_set['Premium Amount']
y_pred = best_model.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('Stacking_Hyper_Tuning_03.csv', index=False)